# SISTEMA DE RECOMENDACION BASADO EN EL CONTENIDO

In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

## Preparacion del set de datos para el modelo

In [3]:
columnas = ['title', 'genres', 'id', 'release_year', 'runtime', 'popularity', 'cast', 'crew']
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv", usecols = columnas)


# Criterios para determinar qué features usar en el modelo 

En base al entendimiento del dominio de las preferencias de los consumidores de películas, se implemente un sistema de recomendación basado en la similitud del contenido de los campos Género, Cast y Crew.
Se considera que el comportamiento de los usuarios puede presentar un tendencia a preferir películas del mismo género, de los mismos actores y del mismo director. 

Por otra parte, también se considera que un sistema de recomendación será exitoso si recomienda películas con mejor recibimiento por parte del público, lo que puede ser inferido por el puntaje.
Finalmente, el sistema recomendará por orden de fecha de lanzamiento, comenzando por las películas mas nuevas, que probablmente el usuario aún no haya visto.
 

In [4]:
columnas = ['popularity', 'release_year']  
df_movies = df_movies.sort_values(by=columnas, ascending=False)

# se verificó que el set de datos contenía series, se decide filtar las filas por duración, eliminando las series y los cortos
df_modelo = df_movies[(df_movies['runtime'] == 0.0) | (df_movies['runtime']>40)]


In [5]:
# limpieza de los campos a utilizar

df_movies['genres'] = df_movies['genres'].str.replace("[\[\]',]", "", regex=True)
df_movies['cast'] = df_movies['cast'].str.replace("[\[\]',]", "", regex=True)
df_movies['crew'] = df_movies['crew'].str.replace("[\[\]',]", "", regex=True)


In [6]:
df_movies = df_movies.head(6000)
df_movies.reset_index(drop=True, inplace=True)
df_movies.reset_index(inplace=True)

# Implementación del modelo

In [7]:
selected_features = ['cast','title', 'crew', 'genres']
combined_features = df_movies['cast']+ ' ' + df_movies['title']+ ' ' + df_movies['genres']+ ' ' + df_movies['crew']


In [8]:
vectorizer = TfidfVectorizer()


In [9]:
feature_vectors = vectorizer.fit_transform(combined_features)


In [10]:
similarity = cosine_similarity(feature_vectors)
print(similarity)


[[1.         0.01441144 0.02561005 ... 0.02176836 0.00543687 0.00232415]
 [0.01441144 1.         0.04390941 ... 0.01543458 0.00337302 0.01085458]
 [0.02561005 0.04390941 1.         ... 0.00350186 0.02376122 0.0017517 ]
 ...
 [0.02176836 0.01543458 0.00350186 ... 1.         0.00743264 0.02512097]
 [0.00543687 0.00337302 0.02376122 ... 0.00743264 1.         0.00189326]
 [0.00232415 0.01085458 0.0017517  ... 0.02512097 0.00189326 1.        ]]


In [11]:
# TITULOS
# se guardan los titulos, para poder hacer la búsqueda por nombre

list_of_all_titles = df_movies['title'].tolist()


In [12]:
def buscar_titulo(titulo_pelicula):
    find_close_match = difflib.get_close_matches(titulo_pelicula, list_of_all_titles)
    close_match = find_close_match[0]
    index_of_the_movie = df_movies[df_movies.title == close_match]['index'].values[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
    print('Peliculas sugeridas : \n')
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = df_movies[df_movies.index==index]['title'].values[0]
        if (i<6):
            print(i, '.',title_from_index)
            i+=1


In [13]:
buscar_titulo('The Terminator')

Peliculas sugeridas : 

1 . The Terminator
2 . Terminator 2: Judgment Day
3 . Superman: Doomsday
4 . Terminator 3: Rise of the Machines
5 . True Lies
